# Dialogue Summarization: Baseline vs System Prompt

This notebook compares Llama's performance on dialogue summarization with and without a system prompt. We'll evaluate using ROUGE scores to measure summary quality.


In [1]:
! pip install -q torch datasets peft huggingface_hub evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.5 MB/s eta 0:00:00


In [18]:
import os
import warnings
import torch
import evaluate
from tqdm import tqdm
from typing import Optional
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from huggingface_hub import login
from datasets import Dataset, load_dataset


warnings.filterwarnings("ignore")



## 2. Authentication and Configuration

Before we begin training, we need to authenticate with HuggingFace (to download models and upload results) and optionally with Weights & Biases (to track training progress).

**Important:** Make sure you have a `.env` file with:
- `HF_TOKEN`: Your HuggingFace access token
- `HF_USERNAME`: Your HuggingFace username

We'll also load our training configuration from `config.json`, which contains all hyperparameters and settings.


In [10]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
HF_USERNAME = userdata.get('HF_USERNAME')

login(HF_TOKEN)


## Load Data from HuggingFace

In [11]:
dataset = load_dataset("knkarthick/samsum")
train_data = dataset["train"]

test_data = dataset['test'].shuffle(seed=42).select(range(200))
val_data = dataset['validation'].shuffle(seed=42).select(range(200))

## Helper Methods for Evaluation

In [15]:
def calculate_rouge(generated_texts, true_summary):
  rouge = evaluate.load("rouge")
  results = rouge.compute(predictions=generated_texts, references=true_summary)
  return results

# Main evaluation function
def evaluate_model(pipe, system_prompt: str = None, num_samples: int = None, batch_size: int = 8):
    """Evaluate model accuracy on the validation data."""

    correct = 0
    correct_index = []
    generated_answers = []
    truncated_count = 0  # Track truncated responses
    token_lengths = []  # Track token lengths

    # Select subset if specified
    eval_data = val_data if num_samples is None else val_data.select(range(num_samples))

    # Prepare prompts and answers
    prompts = [
        f"{system_prompt}\n\n## Dialogue:\n{ex['dialogue']}\n## Summary:\n"
        if system_prompt else f"## Dialogue:\n{ex['dialogue']}\n## Summary:\n"
        for ex in eval_data
    ]
    gt_answers = [ex["summary"] for ex in eval_data]

    # Convert to Hugging Face Dataset for efficient batching
    dataset = Dataset.from_dict({"prompt": prompts})

    print(f"Running evaluation on {len(prompts)} samples...")

    # Run inference in efficient batches
    total_samples = len(dataset)
    from tqdm.auto import tqdm
    pbar = tqdm(total=total_samples, desc="Evaluating", leave=True, position=0)

    for start in range(0, total_samples, batch_size):
        end = min(start + batch_size, total_samples)
        batch_prompts = dataset["prompt"][start:end]
        batch_answers = gt_answers[start:end]

        outputs = pipe(batch_prompts, max_new_tokens=256, batch_size=batch_size, return_full_text=False)

        for j, output in enumerate(outputs):
            generated_only = output[0]["generated_text"]
            generated_answers.append(generated_only)

            # Check token length
            tokens = tokenizer.encode(generated_only)
            token_lengths.append(len(tokens))

            # Check if response was truncated (hit the max_new_tokens limit)
            if len(tokens) >= 256:
                truncated_count += 1
            pbar.update(1)
            pbar.set_postfix({
                "truncated": truncated_count
            })

    pbar.close()
    rouge = calculate_rouge(generated_answers, gt_answers)
    print(f'rouge1: {rouge['rouge1']:.2f}')
    print(f'rouge2: {rouge['rouge2']:.2f}')
    print(f'rougeL: {rouge['rougeL']:.2f}')
    print(f'rougeLsum: {rouge['rougeLsum']:.2f}')
    return rouge

## Load the Model

In [17]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## Baseline Evaluation (No System Prompt)

First, we evaluate the model without any system prompt. This gives us a baseline performance to compare against.


In [41]:
sample = '## Dialogue:\n' + val_data[0]['dialogue'] + '## Summary:\n'
sample

"## Dialogue:\nVictoria: God I'm really broke, I spent way to much this month 😫\nVictoria: At least we get paid soon..\nMagda: Yeah, don't remind me, I know the feeling\nMagda: I just paid my car insurance, I feel robbed 😂\nVictoria: Thankfully mine is paid for the rest of the year 🙏\nMagda: 👌## Summary:\n"

In [54]:
pipe1 = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    device_map="auto",
    temperature=0.1,
    do_sample=False
)
rouge = evaluate_model(
    pipe=pipe1,
    num_samples=200,
    system_prompt=None,
)
torch.cuda.empty_cache()
rouge

Device set to use cuda:0


Running evaluation on 200 samples...


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]

rouge1: 0.14
rouge2: 0.05
rougeL: 0.11
rougeLsum: 0.12


{'rouge1': np.float64(0.14184660607330535),
 'rouge2': np.float64(0.050886601353260966),
 'rougeL': np.float64(0.11154756312691896),
 'rougeLsum': np.float64(0.11929198310863126)}

## Evaluation with System Prompt

Now we evaluate the same model with a system prompt that provides specific instructions for summarization. This helps the model understand the task better and should improve performance.


In [55]:
system_prompt = """
Summarize the dialogue briefly and objectively.
Keep it under three sentences. Include only the main topic, key decisions or actions, and any clear outcome.
Exclude greetings, filler, and minor details.
""".strip()

rouge = evaluate_model(
    pipe=pipe1,
    num_samples=200,
    system_prompt=system_prompt,
)
torch.cuda.empty_cache()
rouge

Running evaluation on 200 samples...


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]

rouge1: 0.19
rouge2: 0.06
rougeL: 0.15
rougeLsum: 0.16


{'rouge1': np.float64(0.1898388418929564),
 'rouge2': np.float64(0.063012614288075),
 'rougeL': np.float64(0.15131574211994864),
 'rougeLsum': np.float64(0.15593718047873387)}

## Example Generation

Let's look at a specific example to see how the model performs with and without the system prompt.


In [47]:
dialogue = val_data['dialogue'][0]
sample = f"## Dialogue:\n{dialogue}\n## Summary:\n"
sample_with_system_prompt = f"{system_prompt}\n\n## Dialogue:\n{dialogue}\n## Summary:\n"
sample

"## Dialogue:\nVictoria: God I'm really broke, I spent way to much this month 😫\nVictoria: At least we get paid soon..\nMagda: Yeah, don't remind me, I know the feeling\nMagda: I just paid my car insurance, I feel robbed 😂\nVictoria: Thankfully mine is paid for the rest of the year 🙏\nMagda: 👌\n## Summary:\n"

In [48]:
val_data['summary'][0]

'Magda and Victoria feel broke. '

In [49]:
pipe1([sample], max_new_tokens=256, return_full_text=False)

[[{'generated_text': "Victoria and Magda are discussing their financial struggles. Victoria is feeling broke and has spent a lot of money this month, while Magda is also feeling the financial strain. They both mention that they get paid soon, which gives them some hope for their financial future. They also joke about Magda's car insurance being paid for the rest of the year, which adds to their sense of relief.\n\n## Context:\nThis dialogue is likely taking place in a casual, everyday conversation between two friends, Victoria and Magda. They may be discussing their personal finances, sharing tips and advice, and offering support and encouragement to each other. The tone of the conversation is lighthearted and humorous, with a touch of sarcasm and playful teasing.\n\n## Themes:\nThe themes of this dialogue include:\n\n* Financial stress and anxiety\n* The importance of having a support system\n* The value of humor and laughter in coping with difficult situations\n* The idea that even s

In [50]:
pipe1([sample_with_system_prompt], max_new_tokens=256, return_full_text=False)

[[{'generated_text': 'Victoria and Magda discuss their financial struggles, with Victoria expressing frustration about overspending. They commiserate about their financial woes, with Magda joking about being "robbed" of her car insurance payment. They both feel relieved that their financial burdens are being alleviated.\n\n## Step 1: Identify the main topic of the dialogue.\nThe main topic is Victoria and Magda\'s financial struggles.\n\n## Step 2: Determine the key decisions or actions.\nVictoria decides to express her frustration about overspending, while Magda decides to joke about being "robbed" of her car insurance payment.\n\n## Step 3: Identify the clear outcome of the dialogue.\nThe clear outcome is that Victoria and Magda feel relieved that their financial burdens are being alleviated.\n\nThe final answer is: Victoria and Magda feel relieved that their financial burdens are being alleviated.'}]]